In [5]:
import polars as pl
import pandas as pd
import polars.datatypes as T

import sys
sys.path.insert(0, 'C:\\Users\\rockh\\Repositories\\Tsuro\\')

from tsuro.datasets import load_cba_trades
from tsuro.utils import cast_strings_to_datetime

In [6]:
trades_df = load_cba_trades(lazy=False)
trades_df

,time,price,volume,dollar_value,market,condition
i64,str,f64,i64,f64,str,str
0,"""6/10/2021 17:00""",100.08,12904,1.2914e6,"""ASX""","""ET XT"""
1,"""6/10/2021 16:54""",100.08,19280,1929542.4,"""ASX""","""ET XT"""
2,"""6/10/2021 16:47""",100.3127,18517,1.8575e6,"""CXA""","""SX XT"""
3,"""6/10/2021 16:46""",100.08,5364,536829.12,"""CXA""",null
4,"""6/10/2021 16:45""",100.08,24624,2.4644e6,"""ASX""","""ET XT"""
…,…,…,…,…,…,…
66572,"""6/10/2021 7:05""",95.01,6000,570060.0,"""ASX""","""EC XT"""
66573,"""6/10/2021 7:05""",94.01,1200,112812.0,"""ASX""","""EC"""
66574,"""6/10/2021 7:05""",94.01,18700,1.757987e6,"""ASX""","""EC XT"""


In [7]:
# trades_df = trades_df.with_columns(
#     pl.col("time").str.strptime("%Y/%m/%d %H:%M:%S", strict = False).alias("date"),
#     pl.col("time").str.strptime(dtype = pl.Datetime, format = "%m/%d/%Y %H:%M").alias("time_new")
# )
trades_df = cast_strings_to_datetime(trades_df, "time", format = "%m/%d/%Y %H:%M")

trades_df

dtypes true
<class 'dict'>


,time,price,volume,dollar_value,market,condition
i64,datetime[μs],f64,i64,f64,str,str
0,2021-06-10 17:00:00,100.08,12904,1.2914e6,"""ASX""","""ET XT"""
1,2021-06-10 16:54:00,100.08,19280,1929542.4,"""ASX""","""ET XT"""
2,2021-06-10 16:47:00,100.3127,18517,1.8575e6,"""CXA""","""SX XT"""
3,2021-06-10 16:46:00,100.08,5364,536829.12,"""CXA""",null
4,2021-06-10 16:45:00,100.08,24624,2.4644e6,"""ASX""","""ET XT"""
…,…,…,…,…,…,…
66572,2021-06-10 07:05:00,95.01,6000,570060.0,"""ASX""","""EC XT"""
66573,2021-06-10 07:05:00,94.01,1200,112812.0,"""ASX""","""EC"""
66574,2021-06-10 07:05:00,94.01,18700,1.757987e6,"""ASX""","""EC XT"""


In [26]:
val = trades_df.select(["time"])[0].item()
val

'6/10/2021 17:00'